# HIPÓTESIS:

##### Importo bibliotecas y el archivo csv

In [2]:
import pandas as pd
import numpy as np
import re
df = pd.read_csv("input/attacks.csv",encoding='ISO-8859-1')
df.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


##### Limpio el DF corrigiendo errores para facilitar el análisis de los datos

In [3]:
print(df.shape)
df.columns

(25723, 24)


Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

##### Corrijo los nombres de las columnas

In [5]:
df_col = df.rename(columns={'Sex ':'Sex','Species ':'Species','Fatal (Y/N)':'Fatal'})

######  Analizo los elementos nulos de cada columna

In [6]:
null_cols = df_col.isnull().sum()

##### Elimino las columnas que no necesito
- Elimino las columnas 'Unnamed:22' y 'Unnamed:23' porque sus valores son 'Nan'.
- Elimino 'href formula', 'href', 'pdf', 'Investigator or source' y 'original order' porque no considero que su contenido aporte información para mi análisis.
- Elimino la columna 'pdf' ya que sus valores son los nombres de los archivos pero no puedo acceder a ellos.
- Elimino 'Case Number', 'Case Numer.1' y 'Case Number.2' porque contienen la misma info que podemos encontrar en 'Date'

In [10]:
clean1 = df_col.drop(columns=['Case Number','Unnamed: 22', 'Unnamed: 23', 'Investigator or Source','pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'Species', 'original order'])
clean1.shape

(25723, 13)

In [11]:
clean1['Fatal'] = clean1['Fatal'].str.replace(' N', 'N')
clean1 = clean1[clean1["Sex"].isin(['M', 'F'])]
clean1 = clean1[clean1.Year != 0]
clean2 = clean1

In [12]:
clean2['Date'] = clean1['Date'].str.replace('-', ' ')
clean2['Date'] = clean1['Date'].str.replace('\d+', '')
clean2['Date'] = clean1['Date'].str.replace('Reported', '')
clean2['Date'] = clean1['Date'].str.replace('Early', '')
clean2['Date'] = clean1['Date'].str.replace('Late', '')
clean2['Date'] = clean1['Date'].str.replace(' ', '')
clean2['Date'] = clean1['Date'].str.replace('Summer, summer, summerof ', '')

In [18]:
clean3 = clean2[clean2.Date != ""]
clean3 = clean3[clean3["Date"].isin(['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug', 'Sep','Oct','Nov','Dec'])]
clean3 = clean3.rename(columns={'Date':'Month'})

##### DF limpio
Elimino columnas que no voy a necesitar para el análisis y me quedo con un nuevo DF limpio

In [17]:
df_clean = clean3.drop(columns=['Name','Age','Time'])
df_clean

,Month,Year,Type,Country,Area,Location,Activity,Sex,Injury,Fatal
0,Jun,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,F,"No injury to occupant, outrigger canoe and pad...",N
1,Jun,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,F,Minor injury to left thigh,N
2,Jun,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,M,Injury to left lower leg from surfboard skeg,N
3,Jun,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,M,Minor injury to lower leg,N
4,Jun,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,M,Lacerations to leg & hand shark PROVOKED INCIDENT,N
...,...,...,...,...,...,...,...,...,...,...
6150,Oct,1753.0,Unprovoked,JAMAICA,NaN,Kingston Harbor,Fell overboard,M,FATAL,Y
6151,Jul,1751.0,Unprovoked,USA,Massachusetts,NaN,Swimming,M,FATAL,Y
6155,Dec,1742.0,Unprovoked,NaN,NaN,Carlisle Bay,Swimming,M,FATAL,Y
6156,Apr,1738.0,Unprovoked,ITALY,Sicily,Strait of Messina,Swimming,M,FATAL,Y


##### Elimino todas las filas con valores null o nan

In [48]:
df_clean2 = df_clean1.dropna()
df_clean2.shape

(2138, 14)

In [15]:
df_clean3["Year"].describe()

count    5205.000000
mean     1971.656292
std        43.995663
min      1703.000000
25%      1950.000000
50%      1986.000000
75%      2007.000000
max      2018.000000
Name: Year, dtype: float64